
<div style="text-align: center; line-height: 0; padding-top: 9px;">
  <img src="https://databricks.com/wp-content/uploads/2018/03/db-academy-rgb-1200px.png" alt="Databricks Learning">
</div>



# LAB - Build a  Feature Engineering Pipeline

Welcome to the "Build a Feature Engineering Pipeline" lab! In this hands-on session, we'll dive into the essential steps of creating a robust feature engineering pipeline. From data loading and preparation to fitting a pipeline and saving it for future use, this lab equips you with fundamental skills in crafting efficient and reproducible machine learning workflows. Let's embark on the journey of transforming raw data into meaningful features for predictive modeling.

**Lab Outline**

+ **Task 1:** Load Dataset and Data Preparation
  + **1.1.** Load Dataset
  + **1.2.** Data Preparation
+ **Task 2:** Split Dataset
+ **Task 3:** Create Pipeline for Data Imputation and Transformation
+ **Task 4:** Fit the Pipeline
+ **Task 5:** Show Transformation Results
+ **Task 6:** Save Pipeline




## REQUIRED - SELECT CLASSIC COMPUTE
Before executing cells in this notebook, please select your classic compute cluster in the lab. Be aware that **Serverless** is enabled by default.

Follow these steps to select the classic compute cluster:
1. Navigate to the top-right of this notebook and click the drop-down menu to select your cluster. By default, the notebook will use **Serverless**.

2. If your cluster is available, select it and continue to the next cell. If the cluster is not shown:

   - Click **More** in the drop-down.
   
   - In the **Attach to an existing compute resource** window, use the first drop-down to select your unique cluster.

**NOTE:** If your cluster has terminated, you might need to restart it in order to select it. To do this:

1. Right-click on **Compute** in the left navigation pane and select *Open in new tab*.

2. Find the triangle icon to the right of your compute cluster name and click it.

3. Wait a few minutes for the cluster to start.

4. Once the cluster is running, complete the steps above to select your cluster.


## Requirements

Please review the following requirements before starting the lesson:

* To run this notebook, you need to use one of the following Databricks runtime(s): **16.3.x-cpu-ml-scala2.12**


## Lab Setup

Before starting the Lab, run the provided classroom setup script. This script will establish necessary configuration variables tailored to each user. Execute the following code cell:

In [0]:
%run ../Includes/Classroom-Setup-2.3

Note: you may need to restart the kernel using %restart_python or dbutils.library.restartPython() to use updated packages.


Using catalog dbacademy and schema labuser10490266_1751005514.


**Other Conventions:**

Throughout this demo, we'll refer to the object `DA`. This object, provided by Databricks Academy, contains variables such as your username, catalog name, schema name, working directory, and dataset locations. Run the code block below to view these details:

In [0]:
print(f"Username:          {DA.username}")
print(f"Catalog Name:      {DA.catalog_name}")
print(f"Schema Name:       {DA.schema_name}")
print(f"Working Directory: {DA.paths.working_dir}")
print(f"Dataset Location:  {DA.paths.datasets}")

Username:          labuser10490266_1751005514@vocareum.com
Catalog Name:      dbacademy
Schema Name:       labuser10490266_1751005514
Working Directory: /Volumes/dbacademy/ops/labuser10490266_1751005514@vocareum_com
Dataset Location:  NestedNamespace (telco='/Volumes/dbacademy_telco/v01', cdc_diabetes='/Volumes/dbacademy_cdc_diabetes/v01')


## Task 1: Load Dataset and Data Preparation


**1.1. Load Dataset:**
+ Load a dataset with features that require imputation and transformation
+ Display basic information about the dataset (e.g., schema, summary statistics)

**1.2. Data Preparation:**

+ Examine the dataset.
+ Identify and discuss the features that need data preparation.
+ Convert data types: Demonstrate converting data types for selected columns (e.g., String to Int, Int to Boolean).
+ Remove a column: Discuss and remove a column with too many missing values.
+ Remove outliers: Implement a threshold-based approach to remove outlier records for a specific column.
+ Save cleaned dataset as "silver table."

**1.1. Load Dataset:**

+ Load a dataset with features that require imputation and transformation
+ Display basic information about the dataset (e.g., schema, summary statistics)


In [0]:
## Set the path of the dataset
dataset_path = f"{DA.paths.datasets.cdc_diabetes}/cdc-diabetes/diabetes_binary_5050_raw.csv"

## Read the CSV file using the Spark read.csv function
## Set the header parameter to True to indicate that the CSV file has a header
## Set the inferSchema option to True for Spark to automatically detect the data types
## Set the multiLine option to True to ensure that Spark reads multi-line fields properly
cdc_df = spark.read.csv(dataset_path, header=True, inferSchema=True, multiLine=True)

## Display the resulting dataframe
display(cdc_df.limit(500))

Diabetes_binary,HighBP,HighChol,CholCheck,BMI,Smoker,Stroke,HeartDiseaseorAttack,PhysActivity,Fruits,Veggies,hvyalcoholconsump,AnyHealthcare,NoDocbcCost,GenHlth,MentHlth,PhysHlth,DiffWalk,Sex,Age,Education,income
0.0,1.0,-0.025331416143639807,1.0,26.0,0.03466823340188028,0.0,0.0,1.0,0.0,1.0,0.0,0.9557398447219455,null,3.0,5.0,30.0,0.0,1.0,4.0,6.0,94607
0.0,1.0,0.9530066352106947,1.0,26.0,0.952108518477824,1.0,0.0,0.0,1.0,0.0,0.0,0.9997260822173017,null,3.0,null,0.0,0.0,1.0,12.0,6.0,78231
0.0,0.0,-0.03240353395468875,1.0,26.0,-0.0205516928829407,0.0,0.0,1.0,1.0,1.0,0.0,0.9902081912754903,null,1.0,0.0,10.0,0.0,1.0,null,6.0,99310
0.0,1.0,1.045839394869003,1.0,28.0,0.9973389564157131,0.0,0.0,1.0,1.0,1.0,0.0,1.0334621727963877,null,3.0,0.0,3.0,0.0,1.0,null,6.0,99665
0.0,0.0,0.04893014144717448,1.0,29.0,0.9875751658928583,0.0,0.0,1.0,1.0,1.0,0.0,0.9556653785712498,null,2.0,0.0,0.0,0.0,0.0,8.0,5.0,81819
0.0,0.0,-0.007806675685258402,1.0,18.0,-0.027525084372416186,0.0,0.0,null,1.0,1.0,0.0,0.03865379600378114,null,2.0,7.0,0.0,0.0,0.0,null,4.0,65444
0.0,0.0,0.996177802963452,1.0,26.0,1.003705912209312,0.0,0.0,1.0,1.0,1.0,1.0,1.0233702445942157,null,1.0,0.0,0.0,0.0,1.0,13.0,5.0,49569
0.0,0.0,-0.04811420282069387,1.0,31.0,0.9557467821540196,0.0,0.0,0.0,1.0,1.0,0.0,1.0297948846417915,null,4.0,0.0,0.0,0.0,1.0,null,4.0,18870
0.0,0.0,-0.03700689683159349,1.0,32.0,-0.009890958385947902,0.0,0.0,1.0,1.0,1.0,0.0,0.9830639133643301,-0.008459602949843237,3.0,null,0.0,0.0,0.0,3.0,6.0,78247
0.0,0.0,-0.006233001874348465,1.0,27.0,null,0.0,null,null,1.0,1.0,0.0,0.9568335361474692,0.03956067890320384,3.0,0.0,6.0,0.0,1.0,6.0,4.0,22585


**1.2. Data Preparation:**

+ Examine the dataset.
+ Identify the features that need data preparation.
+ Convert data types: Demonstrate converting data types for selected columns (e.g., String to Int, Double to Boolean).

In [0]:
## Convert string columns to integer type
from pyspark.sql.types import IntegerType, BooleanType, StringType, DoubleType
from pyspark.sql.functions import col

## List of string columns to convert: (HighBP, CholCheck, PhysActivity)
string_columns = ["HighBP", "CholCheck", "PhysActivity"]

## Iterate over string columns and cast to integer type
for column in string_columns:
   cdc_df = cdc_df.withColumn(column, col(column).cast(IntegerType()))

## Convert double columns to BooleanType (Diabetes_binary, hvyalcoholconsump)
double_columns = ["Diabetes_binary", "hvyalcoholconsump"]
for column in double_columns:
    cdc_df = cdc_df.withColumn(column, col(column).cast(BooleanType()))

## Print the schema
cdc_df.printSchema()
## Examine the printed schema to verify the changes.

root
 |-- Diabetes_binary: boolean (nullable = true)
 |-- HighBP: integer (nullable = true)
 |-- HighChol: double (nullable = true)
 |-- CholCheck: integer (nullable = true)
 |-- BMI: double (nullable = true)
 |-- Smoker: string (nullable = true)
 |-- Stroke: double (nullable = true)
 |-- HeartDiseaseorAttack: string (nullable = true)
 |-- PhysActivity: integer (nullable = true)
 |-- Fruits: double (nullable = true)
 |-- Veggies: double (nullable = true)
 |-- hvyalcoholconsump: boolean (nullable = true)
 |-- AnyHealthcare: double (nullable = true)
 |-- NoDocbcCost: string (nullable = true)
 |-- GenHlth: double (nullable = true)
 |-- MentHlth: string (nullable = true)
 |-- PhysHlth: double (nullable = true)
 |-- DiffWalk: double (nullable = true)
 |-- Sex: string (nullable = true)
 |-- Age: string (nullable = true)
 |-- Education: double (nullable = true)
 |-- income: integer (nullable = true)



+ **Remove a column with too many missing values.**

In [0]:
from pyspark.sql.functions import col, when, count, concat_ws, collect_list

## First, get the count of missing values per column to create a singleton row DataFrame
missing_cdc_df = cdc_df.select([count(when(col(c).contains('null') | (col(c) == '') | col(c).isNull(), c)).alias(c) for c in cdc_df.columns])

## Define a helper function to transpose the DataFrame for better readability
def TransposeDF(df, columns, pivotCol):
    """Helper function to transpose Spark DataFrame"""
    columnsValue = list(map(lambda x: str("'") + str(x) + str("',") + str(x), columns))
    stackCols = ','.join(x for x in columnsValue)
    df_1 = df.selectExpr(pivotCol, "stack(" + str(len(columns)) + "," + stackCols + ")")\
              .select(pivotCol, "col0", "col1")
    final_df = df_1.groupBy(col("col0")).pivot(pivotCol).agg(concat_ws("", collect_list(col("col1"))))\
                   .withColumnRenamed("col0", pivotCol)
    return final_df

## Transpose the missing_cdc_df for better readability
missing_df_T = TransposeDF(spark.createDataFrame([{"Column": "Number of Missing Values"}]).join(missing_cdc_df), missing_cdc_df.columns, "Column")

## Display the count of missing values per column
display(missing_cdc_df)

## Set a threshold for missing data to drop columns
per_thresh = 0.6

## Calculate the total count of rows in the DataFrame
N = cdc_df.count()

## Identify columns with more than the specified percentage of missing data
to_drop_missing = [x.asDict()['Column'] for x in missing_df_T.select("Column").where(col("Number of Missing Values") / N >= per_thresh).collect()]

## Drop columns with more than 60% missing data
print(f"Dropping columns {to_drop_missing} with more than {per_thresh * 100}% missing data")
cdc_no_missing_df = cdc_df.drop(*to_drop_missing)

## Display the DataFrame after dropping columns with excessive missing data
display(cdc_no_missing_df.limit(500))

Diabetes_binary,HighBP,HighChol,CholCheck,BMI,Smoker,Stroke,HeartDiseaseorAttack,PhysActivity,Fruits,Veggies,hvyalcoholconsump,AnyHealthcare,NoDocbcCost,GenHlth,MentHlth,PhysHlth,DiffWalk,Sex,Age,Education,income
0,14071,0,3537,0,21188,0,21011,14183,0,0,0,0,49387,0,7044,0,0,7192,14003,0,0


Dropping columns ['NoDocbcCost'] with more than 60.0% missing data


Diabetes_binary,HighBP,HighChol,CholCheck,BMI,Smoker,Stroke,HeartDiseaseorAttack,PhysActivity,Fruits,Veggies,hvyalcoholconsump,AnyHealthcare,GenHlth,MentHlth,PhysHlth,DiffWalk,Sex,Age,Education,income
false,1,-0.025331416143639807,1,26.0,0.03466823340188028,0.0,0.0,1,0.0,1.0,false,0.9557398447219455,3.0,5.0,30.0,0.0,1.0,4.0,6.0,94607
false,1,0.9530066352106947,1,26.0,0.952108518477824,1.0,0.0,0,1.0,0.0,false,0.9997260822173017,3.0,null,0.0,0.0,1.0,12.0,6.0,78231
false,0,-0.03240353395468875,1,26.0,-0.0205516928829407,0.0,0.0,1,1.0,1.0,false,0.9902081912754903,1.0,0.0,10.0,0.0,1.0,null,6.0,99310
false,1,1.045839394869003,1,28.0,0.9973389564157131,0.0,0.0,1,1.0,1.0,false,1.0334621727963877,3.0,0.0,3.0,0.0,1.0,null,6.0,99665
false,0,0.04893014144717448,1,29.0,0.9875751658928583,0.0,0.0,1,1.0,1.0,false,0.9556653785712498,2.0,0.0,0.0,0.0,0.0,8.0,5.0,81819
false,0,-0.007806675685258402,1,18.0,-0.027525084372416186,0.0,0.0,null,1.0,1.0,false,0.03865379600378114,2.0,7.0,0.0,0.0,0.0,null,4.0,65444
false,0,0.996177802963452,1,26.0,1.003705912209312,0.0,0.0,1,1.0,1.0,true,1.0233702445942157,1.0,0.0,0.0,0.0,1.0,13.0,5.0,49569
false,0,-0.04811420282069387,1,31.0,0.9557467821540196,0.0,0.0,0,1.0,1.0,false,1.0297948846417915,4.0,0.0,0.0,0.0,1.0,null,4.0,18870
false,0,-0.03700689683159349,1,32.0,-0.009890958385947902,0.0,0.0,1,1.0,1.0,false,0.9830639133643301,3.0,null,0.0,0.0,0.0,3.0,6.0,78247
false,0,-0.006233001874348465,1,27.0,null,0.0,null,null,1.0,1.0,false,0.9568335361474692,3.0,0.0,6.0,0.0,1.0,6.0,4.0,22585


+ **Remove outliers: Implement a threshold-based approach to remove outlier records for a specific column.**

In [0]:
## Define cutoff values
MentHlth_cutoff = 0  ## Assuming MentHlth cannot be negative
BMI_cutoff = 50  ## Reasonable upper limit for BMI

## Apply both filters in a single step
cdc_no_outliers_df = cdc_no_missing_df.filter(
    (col("MentHlth") >= MentHlth_cutoff) & 
    (col("BMI") <= BMI_cutoff)
)

## Display the count before and after removing outliers
print(f"Before removing outliers: {cdc_no_missing_df.count()}")
print(f"After removing outliers: {cdc_no_outliers_df.count()}")

## Display the DataFrame after removing outliers
display(cdc_no_outliers_df.limit(500))

Before removing outliers: 70692
After removing outliers: 58443


Diabetes_binary,HighBP,HighChol,CholCheck,BMI,Smoker,Stroke,HeartDiseaseorAttack,PhysActivity,Fruits,Veggies,hvyalcoholconsump,AnyHealthcare,GenHlth,MentHlth,PhysHlth,DiffWalk,Sex,Age,Education,income
false,1,-0.025331416143639807,1,26.0,0.03466823340188028,0.0,0.0,1,0.0,1.0,false,0.9557398447219455,3.0,5.0,30.0,0.0,1.0,4.0,6.0,94607
false,0,-0.03240353395468875,1,26.0,-0.0205516928829407,0.0,0.0,1,1.0,1.0,false,0.9902081912754903,1.0,0.0,10.0,0.0,1.0,null,6.0,99310
false,1,1.045839394869003,1,28.0,0.9973389564157131,0.0,0.0,1,1.0,1.0,false,1.0334621727963877,3.0,0.0,3.0,0.0,1.0,null,6.0,99665
false,0,0.04893014144717448,1,29.0,0.9875751658928583,0.0,0.0,1,1.0,1.0,false,0.9556653785712498,2.0,0.0,0.0,0.0,0.0,8.0,5.0,81819
false,0,-0.007806675685258402,1,18.0,-0.027525084372416186,0.0,0.0,null,1.0,1.0,false,0.03865379600378114,2.0,7.0,0.0,0.0,0.0,null,4.0,65444
false,0,0.996177802963452,1,26.0,1.003705912209312,0.0,0.0,1,1.0,1.0,true,1.0233702445942157,1.0,0.0,0.0,0.0,1.0,13.0,5.0,49569
false,0,-0.04811420282069387,1,31.0,0.9557467821540196,0.0,0.0,0,1.0,1.0,false,1.0297948846417915,4.0,0.0,0.0,0.0,1.0,null,4.0,18870
false,0,-0.006233001874348465,1,27.0,null,0.0,null,null,1.0,1.0,false,0.9568335361474692,3.0,0.0,6.0,0.0,1.0,6.0,4.0,22585
false,null,0.9845930873838862,1,24.0,1.0213867142043727,0.0,1.0,1,1.0,1.0,false,0.981405419463498,3.0,0.0,4.0,0.0,0.0,12.0,4.0,40433
false,0,0.021139561647241224,1,21.0,0.00220757422056006,0.0,0.0,1,1.0,1.0,false,1.0023090409390163,1.0,0.0,0.0,0.0,1.0,4.0,6.0,90729


+ **Save the cleaned dataset as the "silver table" for further analysis**

In [0]:
cdc_df_full = "cdc_df_full"

## Save as DELTA table (silver)
cdc_df_full_silver = f"{cdc_df_full}_silver"
cdc_no_outliers_df.write.mode("overwrite").option("mergeSchema", True).saveAsTable(cdc_df_full_silver)

## Print the resulting DataFrame
display(cdc_no_outliers_df.limit(500))

Diabetes_binary,HighBP,HighChol,CholCheck,BMI,Smoker,Stroke,HeartDiseaseorAttack,PhysActivity,Fruits,Veggies,hvyalcoholconsump,AnyHealthcare,GenHlth,MentHlth,PhysHlth,DiffWalk,Sex,Age,Education,income
false,1,-0.025331416143639807,1,26.0,0.03466823340188028,0.0,0.0,1,0.0,1.0,false,0.9557398447219455,3.0,5.0,30.0,0.0,1.0,4.0,6.0,94607
false,0,-0.03240353395468875,1,26.0,-0.0205516928829407,0.0,0.0,1,1.0,1.0,false,0.9902081912754903,1.0,0.0,10.0,0.0,1.0,null,6.0,99310
false,1,1.045839394869003,1,28.0,0.9973389564157131,0.0,0.0,1,1.0,1.0,false,1.0334621727963877,3.0,0.0,3.0,0.0,1.0,null,6.0,99665
false,0,0.04893014144717448,1,29.0,0.9875751658928583,0.0,0.0,1,1.0,1.0,false,0.9556653785712498,2.0,0.0,0.0,0.0,0.0,8.0,5.0,81819
false,0,-0.007806675685258402,1,18.0,-0.027525084372416186,0.0,0.0,null,1.0,1.0,false,0.03865379600378114,2.0,7.0,0.0,0.0,0.0,null,4.0,65444
false,0,0.996177802963452,1,26.0,1.003705912209312,0.0,0.0,1,1.0,1.0,true,1.0233702445942157,1.0,0.0,0.0,0.0,1.0,13.0,5.0,49569
false,0,-0.04811420282069387,1,31.0,0.9557467821540196,0.0,0.0,0,1.0,1.0,false,1.0297948846417915,4.0,0.0,0.0,0.0,1.0,null,4.0,18870
false,0,-0.006233001874348465,1,27.0,null,0.0,null,null,1.0,1.0,false,0.9568335361474692,3.0,0.0,6.0,0.0,1.0,6.0,4.0,22585
false,null,0.9845930873838862,1,24.0,1.0213867142043727,0.0,1.0,1,1.0,1.0,false,0.981405419463498,3.0,0.0,4.0,0.0,0.0,12.0,4.0,40433
false,0,0.021139561647241224,1,21.0,0.00220757422056006,0.0,0.0,1,1.0,1.0,false,1.0023090409390163,1.0,0.0,0.0,0.0,1.0,4.0,6.0,90729


## Task 2: Split Dataset

**2.1. Split Dataset:**

+ Split the cleaned dataset into training and testing sets in 80:20 ratio

In [0]:
## Split with 80 percent of the data in train_df and 20 percent of the data in test_df
train_df, test_df = cdc_no_outliers_df.randomSplit([0.8, 0.2], seed=42)

## Materialize the split DataFrames as DELTA tables
train_df.write.mode("overwrite").option("overwriteSchema", True).saveAsTable(f"{DA.catalog_name}.{DA.schema_name}.cdc_df_train")
test_df.write.mode("overwrite").option("overwriteSchema", True).saveAsTable(f"{DA.catalog_name}.{DA.schema_name}.cdc_df_baseline")

In [0]:
from pyspark.ml.feature import StandardScaler, RobustScaler, VectorAssembler

## Assuming 'train_data' is your training set DataFrame
feature_columns = ["income"]  # Add your actual feature column names

## Assemble features into a vector
assembler = VectorAssembler(inputCols = feature_columns, outputCol = "income_assembled")
train_assembled_df = assembler.transform(train_df.select(*feature_columns))
test_assembled_df = assembler.transform(test_df.select(*feature_columns))

## Define scaler and fit on the training set
scaler = RobustScaler(inputCol="income_assembled", outputCol= "income_scaled")
scaler_fitted = scaler.fit(train_assembled_df)

## Apply to both training and test sets
train_scaled_df = scaler_fitted.transform(train_assembled_df)
test_scaled_df = scaler_fitted.transform(test_assembled_df)

## Show the resulting DataFrames
print("This is the Training set:")
train_scaled_df.show()
print("This is the Testing set:")
test_scaled_df.show()


This is the Training set:
+------+----------------+--------------------+
|income|income_assembled|       income_scaled|
+------+----------------+--------------------+
| 71331|       [71331.0]|[1.2816638217590512]|
| 56400|       [56400.0]|[1.0133860389902074]|
| 22322|       [22322.0]|[0.40107807025424...|
| 98801|       [98801.0]|[1.7752403198275086]|
| 56109|       [56109.0]|[1.0081573982571197]|
| 91467|       [91467.0]|[1.6434641990836403]|
| 95031|       [95031.0]|[1.7075015721857874]|
| 35459|       [35459.0]|[0.6371215524211661]|
| 29256|       [29256.0]| [0.525667055969814]|
| 24871|       [24871.0]|[0.44687808822208...|
| 18176|       [18176.0]|[0.3265834156859222]|
| 76419|       [76419.0]| [1.373084179319019]|
| 52952|       [52952.0]|[0.9514329350462671]|
| 97530|       [97530.0]|[1.7524031982750874]|
| 85067|       [85067.0]|[1.5284700386308507]|
| 88390|       [88390.0]|[1.5881771628784476]|
| 91007|       [91007.0]| [1.635198993801096]|
| 85830|       [85830.0]|[1.542179

## Task 3: Create Pipeline using Data Imputation and Transformation

**3.1. Create Pipeline:**

+ Create a pipeline with the following tasks:
  + StringIndexer
  + Imputer
  + Scaler
  + One-Hot Encoder


In [0]:
from pyspark.sql.types import IntegerType, BooleanType, StringType, DoubleType
from pyspark.sql.functions import col, count, when

## Get a list of integer & boolean columns
integer_cols = [column.name for column in train_df.schema.fields if (column.dataType == IntegerType() or column.dataType == BooleanType())]

## Loop through integer columns to cast each one to double
## Convert integer columns to double
for column in integer_cols:
    train_df = train_df.withColumn(column, col(column).cast(DoubleType()))
    test_df = test_df.withColumn(column, col(column).cast(DoubleType()))

## Get a list of string, numeric columns
string_cols = [c.name for c in train_df.schema.fields if c.dataType == StringType()]
num_cols = [c.name for c in train_df.schema.fields if c.dataType == DoubleType()]

## Get a list of columns with missing values
## Numerical
num_missing_values_logic = [count(when(col(column).isNull(), column)).alias(column) for column in num_cols]
row_dict_num = train_df.select(num_missing_values_logic).first().asDict()
num_missing_cols = [column for column in row_dict_num if row_dict_num[column] > 0]

## String
string_missing_values_logic = [count(when(col(column).isNull(), column)).alias(column) for column in string_cols]
row_dict_string = train_df.select(string_missing_values_logic).first().asDict()
string_missing_cols = [column for column in row_dict_string if row_dict_string[column] > 0]

## Identify low and high cardinality columns
cardinality_threshold = 100
low_card_cols = []
high_card_cols = []

for col_name in string_cols:
    unique_count = train_df.select(col_name).distinct().count()
    if unique_count <= cardinality_threshold:
        low_card_cols.append(col_name)
    else:
        high_card_cols.append(col_name)

## Print columns with missing values and cardinality info
print(f"Numeric columns with missing values: {num_missing_cols}")
print(f"String columns with missing values: {string_missing_cols}")
print(f"Low-cardinality string columns (OHE allowed): {low_card_cols}")
print(f"High-cardinality string columns (excluded from OHE): {high_card_cols}")

Numeric columns with missing values: ['HighBP', 'CholCheck', 'PhysActivity']
String columns with missing values: []
Low-cardinality string columns (OHE allowed): ['HeartDiseaseorAttack', 'MentHlth', 'Sex', 'Age']
High-cardinality string columns (excluded from OHE): ['Smoker']


In [0]:
## import required libraries
from pyspark.ml import Pipeline
from pyspark.ml.feature import Imputer, VectorAssembler, RobustScaler, StringIndexer, OneHotEncoder

## String/Cat Indexer
## create an additional column to index low-cardinality string columns
## these columns will retain their original null values via 'handleInvalid="keep"'
string_cols_indexed = [c + '_index' for c in low_card_cols]
string_indexer = StringIndexer(inputCols=low_card_cols, outputCols=string_cols_indexed, handleInvalid="keep")

## Imputer (same strategy for all double/indexes)
## create a list of columns containing missing values
## utilize the mode strategy to impute all the missing columns
string_missing_cols_indexed = [c + '_index' for c in string_missing_cols if c in low_card_cols]
to_impute = num_missing_cols + string_missing_cols_indexed

imputer = Imputer(inputCols=to_impute, outputCols=to_impute, strategy='mode')

## Scale numerical
## create a vector of numerical columns as an array in the 'numerical_assembled' column
## robustly scale all the numerical_scaled values for this array in the 'numerical_scaled' column
numerical_assembler = VectorAssembler(inputCols=num_cols, outputCol="numerical_assembled")
numerical_scaler = RobustScaler(inputCol="numerical_assembled", outputCol="numerical_scaled")

## OHE categoricals
## create an OHE encoder to turn the indexed low-cardinality string columns into binary vectors
ohe_cols = [column + '_ohe' for column in low_card_cols]
one_hot_encoder = OneHotEncoder(inputCols=string_cols_indexed, outputCols=ohe_cols, handleInvalid="keep")

## Assembler (All)
## re-collect all columns and create a 'features' column from them
feature_cols = ["numerical_scaled"] + ohe_cols
vector_assembler = VectorAssembler(inputCols=feature_cols, outputCol="features")

## Instantiate the pipeline
## instantiate a pipeline with all the above stages
stages_list = [
    string_indexer,
    imputer,
    numerical_assembler,
    numerical_scaler,
    one_hot_encoder,
    vector_assembler
]

pipeline = Pipeline(stages=stages_list)

##Task 4: Fit the Pipeline
**4.1. Fit the Pipeline:**

+ Use the training dataset to fit the created pipeline.

In [0]:
## Fit the pipeline using the training dataset
pipeline_model = pipeline.fit(train_df)

Uploading artifacts:   0%|          | 0/4 [00:00<?, ?it/s]

##Task 5: Show Transformation Results
**5.1. Transform Datasets:**

+ Apply the fitted pipeline to transform the training and testing datasets.
+ Apply these transformations to different sets (e.g., train, test, validation).

In [0]:
## Transform both the training and test datasets using the previously fitted pipeline model
train_transformed_df = pipeline_model.transform(train_df)
test_transformed_df = pipeline_model.transform(test_df)

## Display the transformed features from the training dataset
display(train_transformed_df.select("features").limit(500))

features
"Map(vectorType -> sparse, length -> 71, indices -> List(1, 2, 4, 7, 10, 11, 14, 15, 16, 20, 53, 60), values -> List(1.0, -0.049999381217593446, 3.375, 1.0, 18.778284439705306, 0.5, 3.0, 1.2816638217590512, 1.0, 1.0, 1.0, 1.0))"
"Map(vectorType -> sparse, length -> 71, indices -> List(1, 2, 4, 7, 10, 11, 14, 15, 16, 22, 52, 64), values -> List(1.0, -0.049963489279271536, 3.25, 1.0, 19.316776700447193, 1.0, 3.0, 1.0133860389902074, 1.0, 1.0, 1.0, 1.0))"
"Map(vectorType -> sparse, length -> 71, indices -> List(1, 2, 4, 10, 11, 12, 14, 15, 18, 20, 53, 58), values -> List(1.0, -0.0497886066568319, 4.25, 19.817137279861846, 1.0, 2.0, 3.0, 0.40107807025424486, 1.0, 1.0, 1.0, 1.0))"
"Map(vectorType -> sparse, length -> 71, indices -> List(1, 2, 4, 7, 10, 11, 14, 15, 16, 20, 53, 56), values -> List(1.0, -0.049787093428877154, 2.875, 1.0, 18.60570059889956, 1.0, 3.0, 1.7752403198275086, 1.0, 1.0, 1.0, 1.0))"
"Map(vectorType -> sparse, length -> 71, indices -> List(1, 2, 4, 10, 11, 14, 15, 17, 20, 53, 57), values -> List(1.0, -0.04977365207419169, 3.875, 19.758934094570943, 0.5, 2.5, 1.0081573982571197, 1.0, 1.0, 1.0, 1.0))"
"Map(vectorType -> sparse, length -> 71, indices -> List(1, 2, 4, 7, 10, 11, 14, 15, 17, 20, 52, 64), values -> List(1.0, -0.049707780272177274, 2.375, 1.0, 20.08146218635358, 0.5, 3.0, 1.6434641990836403, 1.0, 1.0, 1.0, 1.0))"
"Map(vectorType -> sparse, length -> 71, indices -> List(1, 2, 4, 7, 10, 11, 12, 14, 15, 16, 22, 52, 56), values -> List(1.0, -0.04968629708306891, 4.125, 1.0, 19.600652351505705, 1.0, 0.4, 3.0, 1.7075015721857874, 1.0, 1.0, 1.0, 1.0))"
"Map(vectorType -> sparse, length -> 71, indices -> List(1, 2, 4, 7, 10, 11, 12, 14, 15, 16, 20, 54, 56), values -> List(1.0, -0.049654631140334006, 4.375, 1.0, 18.784264259764853, 0.5, 0.2, 2.0, 0.6371215524211661, 1.0, 1.0, 1.0, 1.0))"
"Map(vectorType -> sparse, length -> 71, indices -> List(1, 2, 4, 7, 10, 11, 14, 15, 16, 20, 53, 57), values -> List(1.0, -0.049631702763474755, 3.75, 1.0, 19.545921608356597, 1.0, 3.0, 0.525667055969814, 1.0, 1.0, 1.0, 1.0))"
"Map(vectorType -> sparse, length -> 71, indices -> List(1, 2, 4, 7, 10, 11, 14, 15, 16, 20, 53, 56), values -> List(1.0, -0.04959252521870817, 3.125, 1.0, 18.68605581950237, 1.0, 1.0, 0.44687808822208247, 1.0, 1.0, 1.0, 1.0))"


##Task 6: Save Pipeline
**6.1. Save Pipeline:**

+ Save the fitted pipeline to the working directory.
+ Explore the saved pipeline.

In [0]:
## Save the trained pipeline model to the specified directory in the working directory
pipeline_model.write().overwrite().save(f"{DA.paths.working_dir}/spark_pipelines")

In [0]:
## Load the previously saved pipeline model from the specified directory in the working directory
from pyspark.ml import PipelineModel

## Load the pipeline model
loaded_pipeline = PipelineModel.load(f"{DA.paths.working_dir}/spark_pipelines")

## Display the stages of the loaded pipeline
loaded_pipeline.stages

[StringIndexerModel: uid=StringIndexer_56c23df1ed3c, handleInvalid=keep, numInputCols=4, numOutputCols=4,
 ImputerModel: uid=Imputer_f996156d0d07, strategy=mode, missingValue=NaN, numInputCols=3, numOutputCols=3,
 VectorAssembler_6eb288a237d1,
 RobustScalerModel: uid=RobustScaler_df8171632c7b, numFeatures=16, withCentering=false, withScaling=true,
 OneHotEncoderModel: uid=OneHotEncoder_017ef6c718fc, dropLast=true, handleInvalid=keep, numInputCols=4, numOutputCols=4,
 VectorAssembler_7aa4caa61960]

## Conclusion

In conclusion, this lab demonstrated the crucial steps in preparing and transforming a dataset for machine learning. We covered data cleaning, splitting, and created a pipeline for tasks like imputation and scaling. Saving the pipeline ensures reproducibility, and these foundational concepts can be applied in various machine learning workflows.


&copy; 2025 Databricks, Inc. All rights reserved. Apache, Apache Spark, Spark, the Spark Logo, Apache Iceberg, Iceberg, and the Apache Iceberg logo are trademarks of the <a href="https://www.apache.org/" target="blank">Apache Software Foundation</a>.<br/>
<br/><a href="https://databricks.com/privacy-policy" target="blank">Privacy Policy</a> | 
<a href="https://databricks.com/terms-of-use" target="blank">Terms of Use</a> | 
<a href="https://help.databricks.com/" target="blank">Support</a>